In [10]:
# import essential libraries
import PyPDF2
import re

In [11]:
# extract text from pages
fhand = open('D:\GitHub_Projects\gybn-pb-insights\dataset\PolicyBrief_Nature-based-Solutions.pdf', 'rb')
file_data = PyPDF2.PdfFileReader(fhand, strict=False)
print('Total pages: ', file_data.numPages)

Total pages:  18


In [12]:
page_range = list(range(3, 12)) # pages of interest
raw_data = []
total_chars = []
for page in page_range:
    data = file_data.getPage(page).extractText()
    raw_data.append(data)
    total_chars.append(len(data))

print('Total Characters in Report: ',sum(total_chars))

Total Characters in Report:  24803


## Word-Counter, *Test Runs*

In [13]:
# 1st page 1st line word-counter Test
page1_1 = raw_data[0].split('\n')[0].split()
count_dict = {}
for word in page1_1:
    count_dict[word] = count_dict.get(word, 0) + 1
count_dict

{'3The': 1,
 'term': 1,
 'ﬁNature-based': 1,
 'Solutionsﬂ': 1,
 '(NbS)': 1,
 'started': 1,
 'being': 2,
 'used': 1,
 'in': 1,
 'the': 1,
 'late': 1,
 '2000s,': 1,
 '˜rst': 1,
 'put': 1,
 'forward': 1,
 'by': 1,
 'practitioners': 1,
 'as': 1}

In [14]:
# 1st page all lines word-counter Test
page1 = raw_data[0].split('\n')
page1 = [word for sent in page1 for word in sent.split() if word != '']
counter_dict = {}
for word in page1:
    counter_dict[word] = counter_dict.get(word, 0) + 1
print(f"Common Word : {list(counter_dict.keys())[list(counter_dict.values()).index(max(counter_dict.values()))]}, Counter : {max(counter_dict.values())}")

Common Word : and, Counter : 22


## Word-Counter, *full-report including stop-words*

In [19]:
# All pages all lines word-counter Test
pages = []
for page_num in range(len(raw_data)):
    lines = raw_data[page_num].split('\n')
    page = [word for line in lines for word in line.split() if word != '']
    pages.append(page)

# Word-counter for all pages, excluding stop_words
counter_dict = {}
stop_words = ['The','3The','the','It','it','these','this','This','them','their','and','of','more','which','will','by','as','into','other','also','not','an','be','been','can','to','was','at','or','its','in','on','most','such','that','a','is','for','have','used','are','et','al.,','from','with','has']

for page in pages:
    for word in page:
        if word not in stop_words:
            counter_dict[word] = counter_dict.get(word, 0) + 1
                
max_count = max(counter_dict.values()) # collecting maximum counter
common_word = list(counter_dict.keys())[list(counter_dict.values()).index(max_count)] # Indexing Keys list to identify maximum Value

print(f"Common Word: {common_word}, Counter : {max_count}")

#Important Words --> 'different','NbS','NbS,','Climate','climate','carbon','biodiversity','natural','solutions','Nature-based','Solutions','ecosystem','ecosystems','Ecosystem','global','Global', 'guidelines','principles','International','IUCN','policy','term','sustainable',#'fossil','potential','fuel','approach', 'de˜nition', 'ﬁNature-based','bene˜ts','Solutionsﬂ','help','organizations','funding','under','land','recent','key','emissions','implementation'

Common Word: NbS, Counter : 53


### Identify & Replace irrelevant characters

#### *Test replacement with substitution*

In [16]:
sub_dict = {'˜':'fi','ﬁ':'"','ﬂ':'"'}
sample = ['de˜nition', 'ﬁNature-based','test1','test12','test-mist','bene˜ts','Solutionsﬂ']
for k in sub_dict.keys():
    for s in sample:
        if s.find(k) != -1:
            s_new = re.sub(k, sub_dict[k], s)
            s_ind = sample.index(s)
            s_pop = sample.pop(s_ind)
            print(s_new, s_ind, s_pop)
            sample.insert(s_ind, s_new)
        else:
            continue
print(sample)

definition 0 de˜nition
benefits 5 bene˜ts
"Nature-based 1 ﬁNature-based
Solutions" 6 Solutionsﬂ
['definition', '"Nature-based', 'test1', 'test12', 'test-mist', 'benefits', 'Solutions"']


#### *Implement Test Solution*

In [20]:
# characters from PDF decoding errors by PyPDF2
print(f"=======BEFORE SUBSTITUTION:======\n{pages[0]}\n")

sub_dict = {'˜':'fi','ﬁ':'"','ﬂ':'"'} # substitutes for irrelevant characters

for sub in sub_dict.keys():
    for page in pages:
        for word in page:
            if word.find(sub) != -1:    # search pattern in word
                s_new = re.sub(sub, sub_dict[sub], word) # substitute pattern(key) with substitute(value)
                s_ind = page.index(word) 
                page.remove(word) # remove substituted word
                page.insert(s_ind, s_new)   # insert substitute
            else:   # search pattern in next word
                continue

print(f"=======AFTER SUBSTITUTION:======\n{pages[0]}\n")

=======BEFORE SUBSTITUTION:======
['3The', 'term', 'ﬁNature-based', 'Solutionsﬂ', '(NbS)', 'started', 'being', 'used', 'in', 'the', 'late', '2000s,', 'being', '˜rst', 'put', 'forward', 'by', 'practitioners', 'as', 'the', 'International', 'Union', 'for', 'the', 'Conservation', 'of', 'Nature', 'and', 'the', 'World', 'Bank,', 'and', 'promoted', 'afterwards', 'by', 'policymakers', 'in', 'Europe', '(Eggermont,', 'H.,', 'et', 'al.,', '2015).', 'It', 'was', 'used', 'in', 'the', 'context', 'of', '˜nding', 'new', 'solutions', 'to', 'mitigate', 'and', 'adapt', 'to', 'climate', 'change', 'effects,', 'whilst', 'simultaneously', 'protecting', 'biodiversity', 'and', 'improving', 'sustainable', 'livelihoods.', 'Some', 'of', 'the', 'most', 'widely', 'used', 'de˜nitions', 'are:', 'IUCN', '-', 'International', 'Union', 'for', 'Conservation', 'of', 'Nature', 'ﬁNature-based', 'Solutions', 'are', 'actions', 'to', 'protect,', 'sustainably', 'manage', 'and', 'restore', 'natural', 'and', 'modi˜ed', 'ecosystem

In [18]:
help(list.remove)

Help on method_descriptor:

remove(self, value, /)
    Remove first occurrence of value.
    
    Raises ValueError if the value is not present.

